# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import scipy.stats as st
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt 
from sklearn.metrics import roc_curve
import random as rd

In [2]:
geo_data = pd.read_csv('/datasets/geo_data_0.csv')
geo_data1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
geo_data.head() 

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
geo_data1.head() 

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
geo_data2.head() 

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [6]:
geo_data.info()
print()
geo_data.info()
print()
geo_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Nul

In [7]:
geo_data.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [8]:
geo_data.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [9]:
geo_data2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Данные из 3_х ДФ содержат данные о характеристиках скважин в 3_х регионах. Для каждого региона в ДФ по 10тыс событий. Данные полные, не нуждаются в предобработке. В столбце product данные по всем ДФ распеределны схоже. Про контекст столбцов f0,f1,f2 ничего не изветсно, по этому не буем делать по ним выводы.

Проверим данные на корреляцию.

In [10]:
geo_data.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [11]:
geo_data1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [12]:
geo_data2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Корреляции между данными, которые будем использовать в вобучении не обнаружено, что говорит о том, что ошибка мультиколинеарности нам не грозит.  При этом видно, что целевой признак product во многом зависит от значения f2.

In [13]:
geo_data.duplicated().sum()
geo_data1.duplicated().sum()
geo_data2.duplicated().sum()

0

In [14]:
len(geo_data1['id'].unique())
len(geo_data['id'].unique())
len(geo_data2['id'].unique())

99996

ДФ не имеют явных дубликатов строк, при этом единственный столбец с категориальными данными - id имеет незначительное кол-во повторов. Получатестся, что для одной и той же скважины несколько раз брали пробы с разными показателями. Либо в данные просто вкралась ошибка. посмотрим строки с этими id более детально

In [15]:
duplicated_df = geo_data.pivot_table(index='id', values='f0', aggfunc='count')
duplicated_df.sort_values(by='f0', ascending=False).head(10)

,f0
id,
Tdehs,2
bxg6G,2
QcMuo,2
HZww2,2
TtcGQ,2
AGS9W,2
fiKDv,2
bsk9y,2
A5aEY,2


In [16]:
geo_data[geo_data['id']=='QcMuo']

,id,f0,f1,f2,product
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675


Соверщенно разные показатели для однойго и того же id. В данных скорей всего ошибка. Нельзя утверждать, т.к. нет понимания, что скрыто за f0-f2 и не известен . Учитывая ничтожное кол-во таких подозриельных строк, оставим ДФ как есть.

Т.к. столбец с id не может пригодиться при обучении - избавимся от него

In [17]:
geo_data = geo_data.drop(['id'], axis=1)
geo_data1 = geo_data1.drop(['id'], axis=1)
geo_data2 = geo_data2.drop(['id'], axis=1)

In [18]:
geo_data.info()
print()
geo_data.info()
print()
geo_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f

Данные готовы к работе с моделью

## Обучение и проверка модели

Т.к. имеем 3 ДФ одинаковой конфигурации напишем функцию для подготовки обучающей и тестовой выборок

Поскольку признаки могут быть отображены в разных единицах, для предотвращения возможного искажения результата применим масштабирование

In [19]:
def pre_processing (df):
    features = df.drop('product', axis=1)
    target = df['product']
    f_train, f_valid, t_train, t_valid = train_test_split(features, target, test_size=0.25, random_state=5)
    # проведем масштабирование
    index = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(f_train[index])

    f_train[index] = scaler.transform(f_train[index])
    f_valid[index] = scaler.transform(f_valid[index])
    
    return f_train, f_valid, t_train, t_valid   

In [20]:
f_train, f_valid, t_train, t_valid = pre_processing(geo_data)
f_train_1, f_valid_1, t_train_1, t_valid_1 = pre_processing(geo_data1)
f_train_2, f_valid_2, t_train_2, t_valid_2 = pre_processing(geo_data2)

/tmp/ipykernel_93/295275402.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_train[index] = scaler.transform(f_train[index])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

Составим ункцию для моделирования и оцеки модели

In [21]:
def pediction_and_estimation (feature_train, feature_valid, target_train, target_valid):
    model = LinearRegression() 
    model.fit(feature_train, target_train) 
    prediction_v = model.predict(feature_valid) 
    result = mean_squared_error(target_valid, prediction_v)**0.5 #подсчитаем rmse
    return result, pd.Series(prediction_v).mean(), prediction_v

In [22]:
rmse, mean_prediction, prediction_0 = pediction_and_estimation(f_train, f_valid, t_train, t_valid)
print('Средний запас сырья предсказанный моделью для региона 0 составляет:', round(mean_prediction,2),'. RMSE модели', round(rmse,2) )
t_valid.mean()

Средний запас сырья предсказанный моделью для региона 0 составляет: 92.62 . RMSE модели 37.8


92.90319153767678

In [23]:
rmse, mean_prediction, prediction_1 = pediction_and_estimation(f_train_1, f_valid_1, t_train_1, t_valid_1)
print('Средний запас сырья предсказанный моделью для региона 1 составляет:', round(mean_prediction,2),'. RMSE модели', round(rmse,2))
t_valid_1.mean()

Средний запас сырья предсказанный моделью для региона 1 составляет: 68.65 . RMSE модели 0.89


68.6412755150609

In [24]:
rmse, mean_prediction, prediction_2 = pediction_and_estimation(f_train_2, f_valid_2, t_train_2, t_valid_2)
print('Средний запас сырья предсказанный моделью для региона 0 составляет:', round(mean_prediction,2),'. RMSE модели', round(rmse,2) )
t_valid_2.mean()

Средний запас сырья предсказанный моделью для региона 0 составляет: 95.02 . RMSE модели 40.28


95.08866941281924

Для всех 3_х регионов модели предсказивают средние значения близкие к истинным. Наименьшее rmse выявлено для рагиона 1. Вероятно связано с выявленной ранее положительной корреляцией признака f2 и целевого признака.

## Подготовка к расчёту прибыли

In [25]:
research_point = 500
point_for_mining = 200
budget_for_mining = 10000000000/1000000 #бюджет на разработку (млн руб)
earn_per_barrel = 450000/1000000 #доход с 1 тысячи баррелей (млн руб)
alpha = 2.5

In [26]:
budget_for_point = budget_for_mining / point_for_mining

profitability_point = budget_for_point / earn_per_barrel #столько должно быть запасов в одной скважине для окупаемости
profitability_point

111.11111111111111

Исходя из планов разработки и выделенного бюджета для достижения точки безубыточности новые скважины  должны иметь запасы не менее 111 тысяч баррелей нефти, что весомо превышает средние запасы по ретроспективным данным во всех 3_х регионах.

In [27]:
def earn_func (prediction, target):
    prediction_loc = pd.Series(prediction).sort_values(ascending=False)[:200]#отбираем 200 наиболее больших скважин по предсказанию
    target_loc = target.iloc[prediction_loc.index]#отбираем истинные показатели 200 наиболее больших скважин по предсказанию

    return target_loc.sum()*earn_per_barrel # Возвращаем дохоность, рассчитанную на истинных показателях предсказанных скважин 

In [28]:
earn_func(prediction_0, t_valid)

13126.965047737523

In [29]:
earn_func(prediction_1, t_valid_1)

12415.086696681512

In [30]:
earn_func(prediction_2, t_valid_2)

12334.45520917645

## Расчёт прибыли и рисков 

In [64]:
def bootstrap (probabilities_bt, target_bt, budget_for_mining):
    state = np.random.RandomState(5)
    target_bt = target_bt.reset_index(drop=True)
    values = []
    loss_count = 0
    for i in range(1000):
        target_subsample = target_bt.sample(n=500, replace=True, random_state=state)# 
        probs_subsample = probabilities_bt[target_subsample.index] #берем данные о фактических посещениях исходя из сэмплированной выборке
        profit = earn_func(probs_subsample, target_subsample)-budget_for_mining
        values.append(profit) #сохраняем в переменную занчения прибыли, рассчитанные для каждой подвыборки на каждой итерации
        if profit < 0:
            loss_count+=1
    values = pd.Series(values)
    values_mean = values.mean()
    upper = values.quantile(0.975)
    lower = values.quantile(0.025)
    proba_loss = loss_count/len(values)# вычислим вероятность убытков
    mean_earn = values.mean()
    return upper, lower, proba_loss, mean_earn

In [61]:
upper_0, lower_0, proba_loss_0, mean_earn_0 = bootstrap(prediction_0, t_valid, budget_for_mining)
print("Для региона 0 в 95% случаев прибыль будет в интервалах от", lower_0, "до", upper_0, "млн. руб.")
print("Средняя прибыль составит", mean_earn_0, "млн. руб.")
print("Вероятность убытков", proba_loss_0*100, "%.")

Для региона 0 в 95% случаев прибыль будет в интервалах от -75.26584265347476 до 957.5144993893391 млн. руб.
Средняя прибыль составит 457.19486249448335 млн. руб.
Вероятность убытков 4.3 %.


In [62]:
upper_1, lower_1, proba_loss_1, mean_earn_1 = bootstrap(prediction_1, t_valid_1, budget_for_mining)
print("Для региона 1 в 95% случаев прибыль будет в интервалах от", lower_1, "до", upper_1, "млн. руб.")
print("Средняя прибыль составит", mean_earn_1, "млн. руб.")
print("Вероятность убытков", proba_loss_1*100, "%.")

Для региона 0 в 95% случаев прибыль будет в интервалах от 51.02031826070716 до 829.6580445745146 млн. руб.
Средняя прибыль составит 424.2167669657911 млн. руб.
Вероятность убытков 1.2 %.


In [63]:
upper_2, lower_2, proba_loss_2, mean_earn_2 = bootstrap(prediction_2, t_valid_2, budget_for_mining)
print("Для региона 2 в 95% случаев прибыль будет в интервалах от", lower_2, "до", upper_2, "млн. руб.")
print("Средняя прибыль составит", mean_earn_2, "млн. руб.")
print("Вероятность убытков", proba_loss_2*100, "%.")

Для региона 2 в 95% случаев прибыль будет в интервалах от -170.75064164641583 до 856.7295446472378 млн. руб.
Средняя прибыль составит 334.05218387770884 млн. руб.
Вероятность убытков 11.3 %.


В рамках прроекта по разработке модели для выбора локации для разработки месторожденя рассмотрены датасеты с пробами нефти из 3_х регионов. Поставлена задача построить модель линейной регресси, которая по данным о 500 разведанных точках выберет 200 с наибольшей средней прибылью и оценит вероятность убытков. Исходя из условий задачи для разработки подходи только регион номер 1, со средней ожидаемой прибылью 424млн. руб. т.к. только в этом регионе вероятность убытков 1.2%, что подходит под заданный параметр 2.5%